In [1]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_excel("GrunnvannsBorehull_totalt.xlsx", sheet_name="EnergiBrønn",usecols='E,F,G')
df = df.sort_values('boreDato')
datokolonne = 2
pd.to_datetime(df.iloc[:,datokolonne])

7048   1900-01-01
7251   1900-01-01
5524   1900-01-01
2170   1900-01-01
1981   1900-01-01
          ...    
6657          NaT
7697          NaT
7849          NaT
7850          NaT
8657          NaT
Name: boreDato, Length: 9598, dtype: datetime64[ns]

In [9]:
def bronner_hvert_aar_borer(borer,aar):
    #input:     borer:  Datasett med boredata for valgt oppdragstaker, bruk df for alle oppdragstakere
    #           aar:    Året hvor man vil telle opp antall brønner i
    #output:            Datasett med boredata for valgt oppdragstaker og valgt år
    borer = borer.sort_values('boreDato')
    datoer_borer = borer.iloc[:,datokolonne]
    datoer_aar = [date for date in borer.iloc[:,datokolonne] if date.year == aar]
    indeks_aar = []
    for i in range(0,len(datoer_aar)):
        hvor = np.where(datoer_borer == np.datetime64(datoer_aar[i]))
        indeks_aar = np.concatenate((indeks_aar,hvor[0]))
    if len(indeks_aar) >=1:
        bronner_aar_borer = borer.iloc[int(indeks_aar[0]):int(indeks_aar[-1])+1,:]
    else:
        bronner_aar_borer = pd.DataFrame()
    return bronner_aar_borer

alle_2023 = bronner_hvert_aar_borer(df,2023)
alle_2024 = bronner_hvert_aar_borer(df,2024)


In [11]:
unike_2023 = alle_2023.drop_duplicates(subset='oppdragstaker')
unike_2023 = unike_2023.iloc[:,1]
unike_2023 = unike_2023.sort_values()
unike_2023 = unike_2023.reset_index()
unike_2023 = unike_2023.iloc[:,1]

unike_2024 = alle_2024.drop_duplicates(subset='oppdragstaker')
unike_2024 = unike_2024.iloc[:,1]
unike_2024 = unike_2024.sort_values()
unike_2024 = unike_2024.reset_index()
unike_2024 = unike_2024.iloc[:,1]

#print(unike_2024)

unike_totalt = df.drop_duplicates(subset='oppdragstaker')
unike_totalt = unike_totalt.iloc[:,1]
unike_totalt = unike_totalt.sort_values()
unike_totalt = unike_totalt.reset_index(drop=True)
print(unike_totalt)


0                             ABE BORING AS
1                       Ango Brønnboring AS
2     Anleggsdrift brønn og energiboring AS
3             Aqua Brønn og Energiboring as
4             Asker og Bærum Brønnboring AS
                      ...                  
57                        Værås Brønnboring
58                        WB Brønnboring AS
59       ØSTLANDET BRØNN OG ENERGIBORING AS
60                   Østfold Brønnboring AS
61      Østlandet brønn- og energiboring as
Name: oppdragstaker, Length: 62, dtype: object


In [12]:
antall_2023 = []
antall_2024 = []
for i in unike_2023:
    antall_2023.append(len(alle_2023[alle_2023['oppdragstaker'].str.contains(str(i))]))
    antall_2024.append(len(alle_2024[alle_2024['oppdragstaker'].str.contains(str(i))]))
antall_2023 = pd.DataFrame({'Totalt antall brønner i 2023': antall_2023})
antall_2024 = pd.DataFrame({'Totalt antall brønner i 2024': antall_2024})


antall_enkelthusholdning_2023 = []
antall_storre_anlegg_2023 = []
antall_andre_anlegg_2023 = []
for j in unike_2023:
    hver_borer = alle_2023[alle_2023['oppdragstaker'].str.contains(str(j))]
    antall_enkelthusholdning_2023.append(len(hver_borer[hver_borer['brønnBrukOmfang'].str.contains('Enkelthusholdning')]))
    antall_storre_anlegg_2023.append(len(hver_borer[hver_borer['brønnBrukOmfang'].str.contains('Større anlegg')]))
    antall_andre_anlegg_2023.append(len(hver_borer[hver_borer['brønnBrukOmfang'].str.contains('|'.join(['Gårdsbruk', 'Hytte/fritidsbolig', 'Turistnæring', 'Ukjent']))]))

antall_enkelthusholdning_2023 = pd.DataFrame({'Antall brønner for enkelthusholdning i 2023': antall_enkelthusholdning_2023})
antall_storre_anlegg_2023 = pd.DataFrame({'Antall brønner i større anlegg i 2023': antall_storre_anlegg_2023})
antall_andre_anlegg_2023 = pd.DataFrame({'Antall brønner i andre typer anlegg i 2023': antall_andre_anlegg_2023})

In [13]:
ferdig = pd.concat([unike_2023,antall_2023,antall_2024,antall_enkelthusholdning_2023,antall_storre_anlegg_2023,antall_andre_anlegg_2023],axis=1)
ferdig = ferdig.sort_values('Totalt antall brønner i 2024', ascending=False)
ferdig.to_excel('Aktive_bronnborere_24.xlsx')

In [14]:
antall_totalt = []
for i in unike_totalt:
    antall_totalt.append(len(df[df['oppdragstaker'].str.contains(str(i))]))
antall_totalt = pd.DataFrame({'Totalt antall brønner i totalt': antall_totalt})

ferdig = pd.concat([unike_totalt,antall_totalt],axis=1)
ferdig = ferdig.sort_values('Totalt antall brønner i totalt', ascending=False)
#ferdig.to_excel('Aktive_bronnborere_totalt.xlsx')